# 02_InjuryRisk_Model.ipynb — Injury Risk Prediction

**Goal:** Build and evaluate predictive models for football player injury risk based on training, workload, and wellness data.

**Objective:**
- Create engineered features like ACWR, chronic load, fatigue indices.
- Train multiple ML models (Logistic Regression, Random Forest, XGBoost).
- Evaluate on predictive accuracy, interpret key risk drivers.
- Export player-level injury risk predictions for Power BI visualization.


In [ ]:
# --- 1. Imports & setup ---
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import joblib, json, warnings

warnings.filterwarnings('ignore')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

plt.rcParams.update({'figure.figsize': (8,5), 'axes.grid': True})

BASE = Path('.').resolve()
DATA_IN = (BASE / '..' / 'data').resolve()
OUT_MODELS = (BASE / '..' / 'models').resolve()
OUT_REPORTS = (BASE / '..' / 'reports').resolve()
OUT_MODELS.mkdir(parents=True, exist_ok=True)
OUT_REPORTS.mkdir(parents=True, exist_ok=True)

print('DATA_IN:', DATA_IN)


## 2️⃣ Load Data

In [ ]:
summary = pd.read_csv(DATA_IN / 'player_summary_for_model.csv')
train = pd.read_csv(DATA_IN / 'train_features_prepared.csv', parse_dates=['Date'])

print('Summary shape:', summary.shape)
print('Train shape:', train.shape)
summary.head()


## 3️⃣ Feature Engineering

In [ ]:
# Combine training and summary to derive engineered features
df = train.merge(summary[['PlayerID','Position','Age','InjuryProneScore']], on='PlayerID', how='left')

# Sort and create lag/rolling features
df = df.sort_values(['PlayerID','Date']).reset_index(drop=True)
df['Load'] = df['DistanceKM'] + 0.5*(df['DurationMinutes']/60.0)

df['Acute7'] = df.groupby('PlayerID')['Load'].rolling(7, min_periods=1).sum().reset_index(0,drop=True)
df['Chronic28'] = df.groupby('PlayerID')['Load'].rolling(28, min_periods=7).sum().reset_index(0,drop=True)
df['ACWR'] = df['Acute7'] / df['Chronic28'].replace({0:np.nan})

df['AvgHR_Roll7'] = df.groupby('PlayerID')['AvgHeartRate'].rolling(7, min_periods=3).mean().reset_index(0,drop=True)
df['RPE_Roll7'] = df.groupby('PlayerID')['RPE'].rolling(7, min_periods=3).mean().reset_index(0,drop=True)
df['Fatigue_Roll7'] = df.groupby('PlayerID')['FatigueLevel'].rolling(7, min_periods=3).mean().reset_index(0,drop=True)

# Fill forward & clean NaNs
df = df.fillna(method='ffill').fillna(method='bfill').fillna(0)
df.head()


## 4️⃣ Define Injury Target Variable

In [ ]:
# Simulate injury occurrence (1=injured in next 7 days)
df['TargetInjury'] = 0
for pid in df['PlayerID'].unique():
    sub = df[df['PlayerID']==pid].copy()
    inj_days = sub[sub['FatigueLevel']>8].index  # proxy rule for demonstration
    future_idx = [i+3 for i in inj_days if i+3 < len(df)]
    df.loc[future_idx, 'TargetInjury'] = 1

target_rate = df['TargetInjury'].mean()
print(f'Target injury rate: {target_rate:.3f}')


## 5️⃣ Train/Test Split & Scaling

In [ ]:
feature_cols = ['Load','ACWR','AvgHR_Roll7','RPE_Roll7','Fatigue_Roll7','Age','InjuryProneScore']
X = df[feature_cols].copy()
y = df['TargetInjury']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=feature_cols)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=feature_cols)

print('Train size:', X_train.shape, 'Test size:', X_test.shape)


## 6️⃣ Train Models

In [ ]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=RANDOM_STATE),
    'RandomForest': RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, class_weight='balanced'),
    'XGBoost': XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.05, subsample=0.8, random_state=RANDOM_STATE)
}

results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred_proba = model.predict_proba(X_test_scaled)[:,1]
    roc = roc_auc_score(y_test, y_pred_proba)
    ap = average_precision_score(y_test, y_pred_proba)
    results[name] = {'ROC_AUC': roc, 'PR_AUC': ap}
    print(f"{name}: ROC_AUC={roc:.3f}, PR_AUC={ap:.3f}")


## 7️⃣ Model Evaluation & Diagnostics

In [ ]:
best_model_name = max(results.keys(), key=lambda k: results[k]['ROC_AUC'])
best_model = models[best_model_name]

print('Best model:', best_model_name, '→', results[best_model_name])

y_pred_proba = best_model.predict_proba(X_test_scaled)[:,1]
y_pred = (y_pred_proba >= 0.5).astype(int)

print(classification_report(y_test, y_pred, digits=3))

cm = confusion_matrix(y_test, y_pred)
plt.imshow(cm, cmap='Blues')
plt.title(f'Confusion Matrix — {best_model_name}'); plt.xlabel('Predicted'); plt.ylabel('Actual')
plt.colorbar(); plt.show()

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label=f'{best_model_name} (AUC={roc_auc_score(y_test,y_pred_proba):.2f})')
plt.plot([0,1],[0,1],'--',color='gray')
plt.title('ROC Curve'); plt.legend(); plt.show()


## 8️⃣ Feature Importance

In [ ]:
importances = None
if hasattr(best_model, 'feature_importances_'):
    importances = pd.Series(best_model.feature_importances_, index=feature_cols).sort_values(ascending=False)
    plt.barh(importances.index, importances.values)
    plt.title(f'Feature Importance — {best_model_name}')
    plt.gca().invert_yaxis(); plt.show()
else:
    print('No feature_importances_ attribute available.')


## 9️⃣ Predict Injury Risk & Export Scores

In [ ]:
df['PredictedRisk'] = best_model.predict_proba(scaler.transform(df[feature_cols]))[:,1]
df['RiskCategory'] = pd.cut(df['PredictedRisk'], bins=[0,0.3,0.6,1.0], labels=['Low','Medium','High'])

risk_today = df.groupby('PlayerID')[['PredictedRisk']].last().reset_index()
risk_today = risk_today.merge(summary[['PlayerID','Position','Age']], on='PlayerID', how='left')

out_path = OUT_REPORTS / 'InjuryRiskScores.csv'
risk_today.to_csv(out_path, index=False)
print('Saved injury risk scores to:', out_path)


## 🔟 Save Model & Evaluation Summary

In [ ]:
model_path = OUT_MODELS / f'{best_model_name}_injury_model.joblib'
joblib.dump(best_model, model_path)
joblib.dump(scaler, OUT_MODELS / 'scaler_injury.joblib')

summary_report = {
    'Model': best_model_name,
    'ROC_AUC': results[best_model_name]['ROC_AUC'],
    'PR_AUC': results[best_model_name]['PR_AUC'],
    'FeatureCount': len(feature_cols),
    'TargetRate': float(y.mean())
}

with open(OUT_REPORTS / 'injury_model_summary.json','w') as f:
    json.dump(summary_report, f, indent=2)

print('Saved model + summary.')
summary_report
